<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
import sys, os, json

In [2]:
root_folder = "."
output_dir = os.path.join(root_folder, "./model_save_gpt_234/")
data_path = os.path.join(root_folder, "yelp_review_training_dataset.jsonl")

In [3]:
sys.path.append(root_folder)

# Load Finetuned GPT-2 Model

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm import tqdm
import pandas as pd

In [5]:
config = GPT2Config.from_pretrained(output_dir)
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

In [6]:
if torch.cuda.is_available():
  device = "cuda"
  model.cuda()
else:
  device = "cpu"
model.to(device)
print(device)

cuda


# Do a sample review generation

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
model.eval()

prompt = "<|startoftext|> This place was"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = encoded_inputs['input_ids']
mask = encoded_inputs['attention_mask'].to(device)
generated = generated.to(device)
model = model.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=2,
                                attention_mask=mask
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

NameError: name 'encoded_inputs' is not defined

In [ ]:
from data_parsing import load_dataset

In [ ]:
data = load_dataset(data_path)

In [ ]:
reviews234 = [i for i in data if i[1] == 2 or i[1] == 3 or i[1] == 4]

In [ ]:
len(reviews234)

In [ ]:
data_dir = os.path.join(root_folder, "yelp_review_val_dataset_234.csv")

In [ ]:
df = pd.read_csv(data_dir, error_bad_lines=False, sep='\t', header='infer', lineterminator='\n')  
print(df)

In [ ]:
labels_dir = os.path.join(root_folder, "yelp_review_val_dataset_234_labels.csv")

In [ ]:
labels_df = pd.read_csv(labels_dir, error_bad_lines=False, sep='\t', header='infer', lineterminator='\n')  
print(labels_df)

In [ ]:
labels234 = list(labels_df.text)

In [ ]:
reviews234 = list(df.text)

In [ ]:
len(reviews234), len(labels234)

In [ ]:
reviews234[0]

In [ ]:
labels234[0]

# Create data for N number of samples

In [ ]:
new_data = []
num_to_generate = 2
batch_size = 8

In [ ]:
for i in tqdm(range(0, 28000 - batch_size, batch_size)):
  text = reviews234[i % len(labels234) : (i + batch_size) % len(labels234)]
  review = labels234[i % len(labels234) : (i + batch_size) % len(labels234)]
#   prompt = "<|startoftext|>" + " " + ' '.join(text.split()[:10])
  encodings = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
#   generated = torch.tensor([tokenizer.encode("<|startoftext|>" + " " + ' '.join(p.split()[:7]), ) for p in text])
#   generated = generated.to(device)
  model = model.to(device)

  sample_outputs = model.generate(
                                  encodings['input_ids'], 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=num_to_generate,
                                  attention_mask=encodings['attention_mask']
                                  )
    
  assert len(sample_outputs) == batch_size * num_to_generate
  for i, sample_output in enumerate(sample_outputs):
#     print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
    gpt_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=True).strip()
    new_data.append((gpt_output, review[i % num_to_generate]))
  
#   gpt_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=True).strip()
#   new_data.append((gpt_output, review))


In [ ]:
new_data[10], data[10]

In [ ]:
rs = json.dumps(dict(new_data))

In [ ]:
with open(os.path.join(root_folder, 'new_data_234_56000_samples.json'), 'w') as f:
  json.dump(rs, f)